In [3]:
from utils.custom_dataset import YOLO6ChannelDataset
from torch.utils.data import DataLoader

train_dataset = YOLO6ChannelDataset(
    left_img_dir="D:/intezet/Bogi/Yolo/data/images/train/left",
    right_img_dir="D:/intezet/Bogi/Yolo/data/images/train/right",
    label_dir="D:/intezet/Bogi/Yolo/data/labels/train"
)

val_dataset = YOLO6ChannelDataset(
    left_img_dir="D:/intezet/Bogi/Yolo/data/images/val/left",
    right_img_dir="D:/intezet/Bogi/Yolo/data/images/val/right",
    label_dir="dD:/intezet/Bogi/Yolo/data/labels/val"
)

In [8]:
import torch
from torch.utils.data import DataLoader
from ultralytics.nn.tasks import DetectionModel
from types import SimpleNamespace
from tqdm import tqdm
import os

from utils.custom_dataset import YOLO6ChannelDataset  # Assuming it's defined correctly

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 50
batch_size = 8
save_path = "yolov8_custom6.pt"


# --- Collate Function ---
def collate_fn(batch):
    imgs, targets = zip(*batch)
    imgs = torch.stack(imgs, dim=0)
    targets_with_id = []
    for i, t in enumerate(targets):
        if t.numel() == 0:
            continue
        img_idx = torch.full((t.shape[0], 1), i, dtype=t.dtype)
        targets_with_id.append(torch.cat([img_idx, t], dim=1))
    if len(targets_with_id) == 0:
        targets_combined = torch.zeros((0, 6))
    else:
        targets_combined = torch.cat(targets_with_id, dim=0)
    # print(f"targets_combined: {targets_combined}")
    return imgs, targets_combined


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          collate_fn=collate_fn, num_workers=0)

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        collate_fn=collate_fn, num_workers=0)


# --- Model ---
model = DetectionModel(cfg='ultralytics/cfg/models/v8/yolov8.yaml', ch=6, nc=6).to(device)
model.args = SimpleNamespace(box=7.5, cls=0.5, dfl=1.5, reg_max=15)
model.criterion = model.init_criterion()
model.to(device)
# print(f"Expected output channels: {(model.args.reg_max + 1) * 4 + model.nc}")

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# --- Training Loop ---
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
    
    for imgs, targets in pbar:
        # print(f"imgs.shape: {imgs.shape}")
        imgs = imgs.to(device)
        targets = targets.to(device)

        if targets.shape[0] == 0:
            continue

        optimizer.zero_grad()
        preds = model(imgs)
        if isinstance(preds, tuple) or isinstance(preds, list):
            for i, p in enumerate(preds):
                print(f"preds[{i}].shape = {p.shape}")
        else:
            print(f"preds.shape = {preds.shape}")

        batch = {
            "img": imgs,
            "batch_idx": targets[:, 0],
            "cls": targets[:, 1],
            "bboxes": targets[:, 2:]
        }
        
        loss, loss_items = model.loss(batch, preds)  # loss is scalar, loss_items is [box, cls, dfl]
        loss = loss.sum()
        print(f"loss: {loss}")


        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pbar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"📘 Epoch {epoch+1} - Train Loss: {avg_loss:.4f}")

    # --- Validation ---
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for imgs, targets in val_loader:
            imgs = imgs.to(device)
            targets = targets.to(device)

            if targets.shape[0] == 0:
                continue

            preds = model(imgs)
            loss_out = model.loss(preds, targets)
            val_loss += loss_out['loss'].item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"🧪 Epoch {epoch+1} - Val Loss: {avg_val_loss:.4f}")

    # --- Save ---
    if (epoch + 1) % 5 == 0 or (epoch + 1) == epochs:
        torch.save(model.state_dict(), save_path)
        print(f"✅ Model saved to: {save_path}")


Overriding model.yaml nc=80 with nc=6
WARNING no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       896  ultralytics.nn.modules.conv.Conv             [6, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]          

Epoch 1/50:   0%|          | 0/17 [00:00<?, ?it/s]

preds[0].shape = torch.Size([8, 70, 80, 80])
preds[1].shape = torch.Size([8, 70, 40, 40])
preds[2].shape = torch.Size([8, 70, 20, 20])
loss: 576.159423828125


Epoch 1/50:   6%|▌         | 1/17 [00:12<03:13, 12.07s/it, loss=576]

preds[0].shape = torch.Size([8, 70, 80, 80])
preds[1].shape = torch.Size([8, 70, 40, 40])
preds[2].shape = torch.Size([8, 70, 20, 20])
loss: 529.9642944335938


...

Epoch 1/50:  94%|█████████▍| 16/17 [02:01<00:05,  5.67s/it, loss=473]

preds[0].shape = torch.Size([7, 70, 80, 80])
preds[1].shape = torch.Size([7, 70, 40, 40])
preds[2].shape = torch.Size([7, 70, 20, 20])
loss: 361.15185546875


Epoch 1/50: 100%|██████████| 17/17 [02:06<00:00,  7.43s/it, loss=361]

📘 Epoch 1 - Train Loss: 486.2317


ZeroDivisionError: float division by zero

In [1]:
print("Type of targets:", type(targets))
if isinstance(targets, torch.Tensor):
    print("targets.shape:", targets.shape)
    print("targets sample:\n", targets[:5])  # first 5 rows
elif isinstance(targets, list):
    print("targets is a list with length:", len(targets))
    print("First element type:", type(targets[0]))
    if isinstance(targets[0], torch.Tensor):
        print("targets[0].shape:", targets[0].shape)
        print("targets[0] sample:\n", targets[0][:5])
elif isinstance(targets, dict):
    print("targets.keys():", targets.keys())
    for k, v in targets.items():
        print(f"{k}: type={type(v)}, shape={getattr(v, 'shape', 'N/A')}")
else:
    print("Unknown targets format:", targets)

NameError: name 'targets' is not defined